<a href="https://colab.research.google.com/github/Swaraj9/FIRS_Models/blob/main/FIRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Imports
import torch
import torchvision

from torchvision.datasets import Food101
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
#Device Agnostic Code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
!rm -r food-101

In [ ]:
from pathlib import Path
import os

def getData():
  if "food-101" in os.listdir():
    print("Dataset already exists")
  else:
    print("Donloading the data...")
    !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
    print("Dataset Downloaded")
    print("Extracting data...")
    !tar xzf food-101.tar.gz
    print("Extraction done!")

getData()

Donloading the data...
--2023-07-20 08:10:14--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2023-07-20 08:10:14--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz.2’

food-101.tar.gz.2   100%[===================>]   4.65G  22.3MB/s    in 5m 48s  

2023-07-20 08:16:03 (13.7 MB/s) - ‘food-101.tar.gz.2’ saved [4996278331/4996278331]

Dataset Downloaded
Extracting data...

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpect

In [ ]:
from shutil import copy
from collections import defaultdict

def prepareData(filepath, src, dest):
  classesImages = defaultdict(list)
  with open(filepath, 'r') as txt:
    paths = [read.strip() for read in txt.readlines()]
    for p in paths:
      food = p.split('/')
      classesImages[food[0]].append(food[1] + '.jpg') #{apple_pie: [0000, 00001], ...}

  print("Copying Images")
  for food in classesImages.keys():
    if not os.path.exists(os.path.join(dest,food)):
      os.makedirs(os.path.join(dest,food))
    for i in classesImages[food]:
      copy(os.path.join(src,food,i), os.path.join(dest, food, i))
  print("Copying Done!")

In [ ]:
print("Creating train data...")
prepareData("food-101/meta/train.txt", "food-101/images", "food101/train")

print("Creating test data...")
prepareData("food-101/meta/test.txt", "food-101/images", "food101/test")

Creating train data...


FileNotFoundError: ignored

In [ ]:
from shutil import copytree, rmtree
def datasetMini(foodList, src, dest):
  if os.path.exists(dest):
    rmtree(dest)
  os.makedirs(dest)
  for foodItem in foodList:
    print("Copying images into", foodItem)
    copytree(os.path.join(src, foodItem), os.path.join(dest, foodItem))

In [ ]:
foodList = ["samosa", "pizza", "omelette"]
srcTrain = "food101/train"
destTrain = "food101/trainMini"
srcTest = "food101/test"
destTest = "food101/testMini"

print("Creating mini train and test sets...")
datasetMini(foodList, srcTrain, destTrain)
datasetMini(foodList, srcTest, destTest)

In [ ]:
from torchvision import datasets

trainTransforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])
])

testTransforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])
])

trainData = datasets.ImageFolder(root="food101/train", transform=trainTransforms)
testData = datasets.ImageFolder(root="food101/test", transform=testTransforms)
trainDataMini = datasets.ImageFolder(root="food101/trainMini", transform=trainTransforms)
testDataMini = datasets.ImageFolder(root="food101/testMini", transform=testTransforms)

In [ ]:
trainLoader = DataLoader(trainData, 64, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)
testLoader = DataLoader(testData, 64, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)
trainLoaderMini = DataLoader(trainDataMini, 64, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)
testLoaderMini = DataLoader(testDataMini, 64, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)

In [ ]:
import time
from tqdm.auto import tqdm
from google.colab import drive
import shutil

def train(model, trainLoader, testLoader, lossFn, optimizer, scheduler, epochs=25):
  since = time.time()

  for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print('-' * 10)

  # Training the model
    model.train()

    trainLoss, trainAcc = 0, 0

    print(len(trainLoader))
    for batch, (X, y) in tqdm(enumerate(trainLoader)):
      X, y = X.to(device), y.to(device)

      yPred = model(X)

      loss = lossFn(yPred, y)

      optimizer.zero_grad(set_to_none=True)
      loss.backward()
      optimizer.step()

      yPredClass = torch.argmax(torch.softmax(yPred, dim=1), dim=1)
      trainAcc += (yPredClass == y).sum().item()/len(yPred)

    trainAcc /= len(trainLoader)

    #Test
    model.eval()

    testLoss, testAcc = 0, 0

    with torch.inference_mode():
      for batch, (X, y) in tqdm(enumerate(testLoader)):
        X, y = X.to(device), y.to(device)

        testPredLogits = model(X)

        loss = lossFn(testPredLogits, y)
        testLoss += loss.item()

        testPredLabels = testPredLogits.argmax(dim=1)
        testAcc += ((testPredLabels == y).sum().item()/len(testPredLabels))

    testLoss /= len(testLoader)
    testAcc /= len(testLoader)

    print(f"TrainAcc: {trainAcc:.4f} Loss: {testLoss:.4f} Acc: {testAcc:.4f}")

    timeElapsed = time.time() - since
    print(f"Training Complete in {timeElapsed // 60:.0f}m {timeElapsed % 60:.0f}s")

  try:
    if not os.path.exists("models"):
      os.mkdir("models")
    torch.save(model.state_dict(), f"models/{model.name}_E{epochs}_{testAcc:.4f}%.pt")
    drive.mount("/content/drive")
    dest = '/content/drive/MyDrive/models'
    if(os.path.exists(dest)):
      shutil.rmtree(dest)
    shutil.copytree("models",dest)
  except:
    print("Couldnt copy files to google drive")
  return model

In [ ]:
from torch import nn
from torchvision import models

!pip install torchinfo
from torchinfo import summary

model = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
model.name = "vit_b_16"

In [ ]:
model

In [ ]:
for param in model.parameters():
    param.requires_grad = False

numFtrs = model.heads[0].in_features

model.heads = nn.Sequential(nn.Linear(numFtrs, 101, bias = True))
model = model.to(device)

lossFn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
summary(model, (1,3,224,224))

In [ ]:
model = train(model, trainLoader, testLoader, lossFn, optimizer, scheduler, 5)

In [ ]:
!git remote remove origin

In [ ]:
!git remote add origin git@github.com:Swaraj9/FIRS_Models.git

In [ ]:
!git branch -M main

In [ ]:
!git config --global user.email "swaraj.dusane90@gmail.com"
!git config --global user.name "Swaraj9"

In [ ]:
!git add .
!git commit -m "First Commit"

In [ ]:
!git push -u origin main

fatal: could not read Username for 'https://github.com': No such device or address
